In [1]:
import numpy as np
import pandas as pd

In [2]:
# !curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
# conda_installer.install()
conda_installer.install()
# !/root/miniconda/bin/conda info -e
import rdkit
import deepchem as dc

add /home/ali/miniconda/lib/python3.6/site-packages to PYTHONPATH
all packages are already installed


In [3]:
pdb = rdkit.Chem.MolFromPDBFile('../Datasets/PDBBindSmallSet/1a30-ligand.pdb')

In [62]:
ligand = pd.read_csv('../Datasets/PDBBindSmallSet/csv-files/1a94-info.csv')

In [64]:
ligand.round(3).iloc[0]

Unnamed: 0               0.000
x                       67.671
y                        9.696
z                       12.088
atomic-charge           -0.202
atomic-radius            1.550
effective-born-radii     0.409
Name: 0, dtype: float64

In [8]:
rdkit_order = None
for c in pdb.GetConformers():
    rdkit_order = c.GetPositions()

In [10]:
f = None
df = pd.DataFrame(columns=ligand.columns)
for i, atom_pos in enumerate(rdkit_order):
#     print(i, " ", atom_pos)
#     print(ligand['x'].iloc[0])
#     print(ligand[(ligand['x'] == atom_pos[0]) & (ligand['y'] == atom_pos[1]) & (ligand['z'] == atom_pos[2])])
    df = df.append(ligand[(ligand['x'] == atom_pos[0]) & (ligand['y'] == atom_pos[1]) & (ligand['z'] == atom_pos[2])], ignore_index=True)
#     break

In [11]:
featurizer = dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)

In [12]:
c_f = featurizer.featurize(pdb)[0]

In [44]:
import os
pdb_files = ["../Datasets/PDBBindSmallSet/" + pdb for pdb in os.listdir("../Datasets/PDBBindSmallSet") if pdb.__contains__('.pdb')]
atoms_csvs = ["../Datasets/PDBBindSmallSet/csv-files/" + atom_csv for atom_csv in os.listdir("../Datasets/PDBBindSmallSet/csv-files/") if atom_csv.__contains__('.csv')]
physics_df = pd.read_csv('../Datasets/PDBBindSmallSet/polished.csv')

In [38]:
physics_df = physics_df[physics_df['dataset group name'] == 'pdbbind']

In [65]:
atoms_csvs_dict = {atoms_csv.split('/')[-1].split('.')[0].split('-')[0]: pd.read_csv(atoms_csv).round(3) for atoms_csv in atoms_csvs}

In [55]:
mols_dict = {}
for pdb in pdb_files:
    mols_dict.update({pdb.split('/')[-1].split('.')[0].split('-')[0]: rdkit.Chem.MolFromPDBFile(pdb)})

In [104]:
def convertBONDtoOneHotVector(bond_type):
    BOND_DICT = {dir(rdkit.Chem.rdchem.BondType)[i]: i for i in range(22)}
    one_hot = np.zeros(22)
    one_hot[BOND_DICT[str(bond_type)]] = 1
    return one_hot

In [107]:
for mol_name, mol in mols_dict.items():
    mol_f = featurizer.featurize(mol)[0]
    atoms_gb = atoms_csvs_dict[mol_name]
    
    for c in mol.GetConformers():
        rdkit_order = c.GetPositions()
    df = pd.DataFrame(columns=atoms_gb.columns)
    for i, atom_pos in enumerate(rdkit_order):
        df = df.append(atoms_gb[(atoms_gb['x'] == atom_pos[0]) & (atoms_gb['y'] == atom_pos[1]) & (atoms_gb['z'] == atom_pos[2])], ignore_index=True)
    atoms_gb = df
    atom_physics_features = atoms_gb[['atomic-charge', 'atomic-radius', 'effective-born-radii']].to_numpy()
    atom_features = np.concatenate([mol_f.atom_features, atom_physics_features], axis = 1)
    
    adjacency_list = []
    for atom in mol.GetAtoms():
        atom_bonds = []
        for neighbor in atom.GetNeighbors():
            bond_type = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx()).GetBondType()
            neighbor_idx = neighbor.GetIdx()
            atom_bonds.append([neighbor_idx, convertBONDtoOneHotVector(bond_type)])
        adjacency_list.append(atom_bonds)
    

[[1, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0.])], [4, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0.])]]


In [85]:
atom = mols_dict['1a1e'].GetAtomWithIdx(6)
[x.GetIdx() for x in atom.GetNeighbors()]

[1, 7, 8]